In [13]:
import pandas as pd

import geopandas as gpd

import numpy as np


# Radios censales

In [43]:

import json
import urllib

import shapely
import pyproj

p1 = pyproj.Proj(init='epsg:3857') # 'epsg:3395' or 'epsg:3857'
p2 = pyproj.Proj(proj='latlong',datum='WGS84')


In [44]:
# RADIO 62,030,301 is buggy. Big patch in south Bs As

# The following loads a JSON of censal radios and creates a geojson

df_list = []
data_list = []

# Breakup for country-wide query
for start, end in [(0, 25000), (25000, 50000), (50000, 75000)]:

    #total pais
    url = "https://services1.arcgis.com/gKFHG0iKXPnKkg1l/arcgis/rest/services/Radios_censales/FeatureServer/0/query?"\
    +"where=OBJECTID+>=+"+str(start)+"+AND+OBJECTID+<"+str(end)+"&objectIds=&time=&geometry=&geometryType=esriGeometryPolygon&inSR=&spatialRel=esriSpatialRelIntersects&resultType=none&distance=0.0&units=esriSRUnit_Meter&returnGeodetic=false&outFields=*&returnGeometry=true&returnCentroid=false&multipatchOption=xyFootprint&maxAllowableOffset=&geometryPrecision=&outSR=&datumTransformation=&applyVCSProjection=false&returnIdsOnly=false&returnUniqueIdsOnly=false&returnCountOnly=false&returnExtentOnly=false&returnDistinctValues=false&orderByFields=&groupByFieldsForStatistics=&outStatistics=&having=&resultOffset=&resultRecordCount=&returnZ=false&returnM=false&returnExceededLimitFeatures=true&quantizationParameters=&sqlFormat=none&f=pjson&token="

    res = urllib.urlopen(url)
    data = json.loads(res.read())

    # Format for Geojson
    for d in data["features"]:
        d["geometry"]["coordinates"] = d["geometry"].pop("rings")
        # Project
        for i in range(len(d["geometry"]["coordinates"])):
            d["geometry"]["coordinates"][i] = [list(pyproj.transform(p1, p2, point[0], point[1])) for point in d["geometry"]["coordinates"][i]]
        
        # set rounding level according to polygon size
#         bounds = shapely.geometry.Polygon(d["geometry"]["coordinates"][0]).bounds
#         round_lev =  - int(min(np.floor(np.log10(bounds[2] - bounds [0]) - 2.5), 
#                         np.floor(np.log10(bounds[3] - bounds [1]) - 2.5)))

#         for i in range(len(d["geometry"]["coordinates"])):
#             d["geometry"]["coordinates"][i] = np.round(d["geometry"]["coordinates"][i], round_lev).tolist()
       
        d["geometry"]["type"] = "Polygon"
        d["properties"] = d.pop("attributes")
        d["type"] = "Feature"
        
    data_list += [data]
    
    df = pd.DataFrame([e['properties'] for e in data['features']]) 
    df_list += [df]
    
df = pd.concat(df_list).reset_index(drop = True)

data = data_list[0]
if len(data_list) > 1:
    for di in data_list[1:]: 
        for k in di:
            try:
                data[k].update(di[k])
            except AttributeError: # in case it is a list
                data[k] += di[k]

### Save geojson and geodataframe

In [45]:
# Geojson
with open('./Radios_censales/radios.json', 'w') as outfile:
    json.dump(data, outfile)
    
# Geodataframe
gdf = gpd.GeoDataFrame(pd.DataFrame([e['properties'] for e in data['features']]) )
gdf['geometry'] = pd.DataFrame([shapely.geometry.Polygon([tuple(pt) for pt in e['geometry']['coordinates'][0]]) for e in data['features']]) 
gdf.to_file(driver = 'ESRI Shapefile', filename = './Radios_censales/radios_w_geometry.shp')

# g2 = gpd.GeoDataFrame.from_file("./../EPH/radios_w_geometry.shp")


# Departamentos

In [34]:
dptos_gdf = gpd.GeoDataFrame.from_file(driver = 'ESRI Shapefile', filename = './dptos/pxdptodatosok.shp')

dptos_gdf = dptos_gdf.apply(lambda x: pd.to_numeric(x, errors = 'ignore'))

# Delete if geojson exists, cause overwriting is not supported
try:
    os.remove('./dptos/dptos.geojson')
except OSError:
    pass

dptos_gdf.to_file("./dptos/dptos.geojson", driver="GeoJSON")

In [41]:
from mapbox import Uploader
import json

def upload_file(data, name, username = 'matuteiglesias', token = 'sk.eyJ1IjoibWF0dXRlaWdsZXNpYXMiLCJhIjoiY2puODA4bW8xMGV1dzNrcGtiOGp6NXQ5aCJ9.DohKmjn_o6MK1Y4Q5FG8ew'):
    # Dump into file for upload
    with open('./upload_data.geojson', 'w') as outfile:
        json.dump(data, outfile)

    service = Uploader(access_token=token)
    with open('./upload_data.geojson', 'r') as src:
        # Acquisition of credentials, staging of data, and upload
        # finalization is done by a single method in the Python SDK.
        upload_resp = service.upload(src, username+'.'+name)
        
upload_file(data = json.load(open('./dptos/dptos.geojson')), name = 'deptos')
